# EQUATIONS

### Imports

In [2]:
import math
import scipy.constants as spc
from scipy.optimize import brentq
from scipy.optimize import root
import sympy as sp
from brian2 import *
import parameters


### Variables

In [3]:
params = parameters.return_initial_parameters()

F = 9.648534297750000e+04

# params = {
#     'K': 0.693e6,
#     'Na': 2.04e6,
#     'Cl': 50,
# }
# params = {## params to setup reverse potentials
#             'C0_Na_N': 10,
#             'C0_Na_E': 145,
#             'C0_K_N': 130,
#             'C0_K_E': 3,
#             'C0_Cl_N': 5,
#             'C0_Cl_E': 130
# }
# params = {
# }
# params = {
#     'zeta_Na' : 13,
#     'zeta_K' : 0.2
# }

print(params)

{'C0_Na_N': 10. * mmolar, 'C0_Na_E': 145. * mmolar, 'C0_K_N': 130. * mmolar, 'C0_K_E': 3. * mmolar, 'C0_Cl_N': 5. * mmolar, 'C0_Cl_E': 130. * mmolar, 'c_m': 0.01 * metre ** -4 * kilogram ** -1 * second ** 4 * amp ** 2, 'g_Na': 20400. * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_Na_L': nan * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_K_L': 0. * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_K': 6930. * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_Cl_L': 0.5 * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_KCC': 0.62544968 * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'v_thr': -50. * mvolt, 'resting_potential': -70, 'U_m': -38. * mvolt, 'U_h': -66. * mvolt, 'U_n': 18.7 * mvolt, 'W_m': 6. * mvolt, 'W_h': 6. * mvolt, 'W_n': 9.7 * mvolt, 'T_exp': 37, 'I_dc': 0. * metre ** -2 * amp, 'I_ramp': 0. * metre ** -2 * amp, 'I_max': 0.04 * metre ** -2 * amp, 'T_ramp': 100. * second, 'TimedArray': <brian2.input.timedarray.Tim

### Functions

In [ ]:
## Basic functions
def ThermalVoltage(T):
    return 8.314472000000000 *(T+273.15)/F
def NernstPotential(x_e,x_i,z,T):
    return params['V_T']/z*np.log(x_e/x_i)
def Hill(x,K,n):
    return x**n/(x**n+K**n)


## params and ionic variations
def C_Cl_N(C0_Cl_N, n_Cl_N, Volume):
    return C0_Cl_N + n_Cl_N/Volume
def C_Na_N(C0_Na_N, n_Na_N, Volume):
    return C0_Na_N + n_Na_N/Volume
def C_Na_N(C0_K_N, n_K_N, Volume):
    return C0_K_N + n_K_N/Volume
def dn_Na_N(I_Na, I_NKP, surface):
    return -surface/F*(I_Na + 3*I_NKP)
def dn_K_N(I_K, I_NKP, I_KCC, surface):
    return -surface/F*(I_K - 2*I_NKP - I_KCC)    
def dn_Cl_N(I_Cl, I_KCC, surface):
    return surface/F*(I_Cl + I_KCC)

# ## Calculation reversal potential for each ion
# params['E_K'] = NernstPotential(params['C0_K_E'],params['C0_K_N'],1, 37) * 1000
# params['E_Na'] = NernstPotential(params['C0_Na_E'],params['C0_Na_N'],1, 37) * 1000
# params['E_Cl'] = NernstPotential(params['C0_Cl_E'],params['C0_Cl_N'],-1, 37) * 1000


## Calculation gating variables 
def alpha_m(V):
    return 182*(V - (-38 * mvolt))/(1 - math.exp((-38 * mvolt - V)/6/mvolt))
def beta_m(V):
    return 124*(-38 * mvolt - V)/(1 - math.exp((V - (-38 * mvolt))/6/mvolt))
def alpha_h(V):
    return 15*(-66 * mvolt - V)/(1 - math.exp((V - (-66 * mvolt))/6/mvolt))
def beta_h(V):
    return 15*(V - (-66 * mvolt))/(1 - math.exp((-66 * mvolt - V)/6/mvolt))
def gating_variable_m(V):
    return alpha_m(V)/(alpha_m(V) + beta_m(V))
def gating_variable_h(V):
    return alpha_h(V)/(alpha_h(V) + beta_h(V))
def gating_variable_n(V):
    return 1/(1+math.exp((18.7 *mvolt - V)/9.7/mvolt))   

## Calculations ionic currents 1 - VARIABLE VOLTAGE
def I_Cl(V, Reversal_potential): 
    return params['Cl'] * (V - Reversal_potential)
def I_Na(V, Reversal_potential):   
    return params['Na'] * (gating_variable_m(V)**3) * gating_variable_h(V) * (V - Reversal_potential)
def I_K(V, Reversal_potential):    
    return params['K'] * (gating_variable_n(V)) * (V - Reversal_potential)
def sigma():
    return 1/7 * (exp(params['C0_Na_E']/67.3/mmolar)-1)
def f_NaK(V, V_T):  
    return 1/(1 + 0.1245 * exp(-0.1 * V/V_T) + 0.0365 * sigma() * exp(-V/V_T))

def I_NKP(I_NKP_max, f_NaK, params):
    return I_NKP_max * f_NaK  * Hill(params['C0_Na_N'], params['zeta_Na'], 1.5) * Hill(params['C0_K_E'], params['zeta_K'], 1)
def I_KCC(params):
    return params['g_KCC']*(params['E_K'] - params['E_Cl'])



## Calculations ionic currents_inf 2 - AT RESTING POTENTIAL (ik that they aren't necessary with the functions above already being defined but hey)
def I_Na_inf(leakage_conductance, potential):
    return (params['g_Na'] * gating_variable_m(potential)**3 * gating_variable_h(potential) + leakage_conductance) * (potential - params['E_Na'])
def I_Na_inf_ohne_lc():
    return (params['g_Na'] * gating_variable_m(-70)**3 * gating_variable_h(-70)) * (-70 - params['E_Na'])
def I_K_inf(potential):
    return params['g_K'] * gating_variable_n(potential) * (potential - (params['E_K']))
# TODO: include leakage conductance potassium channel
def I_Cl_inf(potential):
    return params['g_Cl_L'] * (potential - params['E_Cl'])

## Calculations I_NKP_max - BASED ON THE RELATION BETWEEN I_Na_inf and I_NKP <- I_Na_inf + 3*I_NKP = 0
def calculate_I_NKP_max(I_Na_inf, f_NaK, Hill_Na, Hill_K):
    return -I_Na_inf/(3 * f_NaK * Hill_Na * Hill_K)

## Consistency equation for the calculation of the necessary leakage conductance of the sodium-channel
def equilibrium_current(potential, leakage_conductance):
    return 2/3 * I_Na_inf(leakage_conductance, potential) + I_K_inf(potential) + I_Cl_inf(potential)


## Calculation of the total currents in HH and HH-ECS
def total_current_hh(leakage_conductance, potential):
    return   - I_K_inf(potential) - I_Na_inf(leakage_conductance, potential) - I_Cl(potential, params['E_Cl'])
# TODO: include dynamic potential for the I_Na_inf
def total_current_hhecs(leakage_conductance, x, potential):
    return   - I_K_inf(potential) - I_Na_inf(leakage_conductance, potential) - I_Cl(potential, params['E_Cl']) - I_NKP(potential, f_NaK(potential, params['V_T'], sigma()), params, params)

## Calculation of the resting state Potential in HH + Calculation conductance of KCC + Calculation leakage conductance of the sodium-channel
# def calculate_resting_state_potential_hh():
#     resting_potential = brentq(total_current_hh, -100, 50)
#     return resting_potential
def conductance_KCC(V, Nernst_Cl, Nernst_K, conductance_Cl):
    return conductance_Cl * (V - Nernst_Cl)/(Nernst_Cl - Nernst_K)
def calc_leakage_conductance():
    f_single_variable = lambda x: equilibrium_current(params['resting_potential'] * mvolt, x * usiemens/cm**2)
    solution = brentq(f_single_variable, 0, 50000) * usiemens/cm**2
    return solution

params['g_KCC'] = conductance_KCC(params['resting_potential']*mvolt, params['E_Cl'], params['E_K'], params['g_Cl_L'])




### Actual Value Calculations

In [5]:
print(params)


# 2 variables missing to achieve equilibrium for the HH-ECS: 
# 1. additional leakage conductance of the sodium-channel
# 2. I_NKP_max for the sodium-potassium pump (derived from the I_Na_inf in this script but also possible from I_K_inf and I_KCC_inf - see 2.27)

potential = -70*mvolt

## Calculations of VALUE 1 (g_Na)
leakage_conductance = calc_leakage_conductance() 
print(f'SODIUM LEAKAGE CONDUCTANCE: {leakage_conductance}')

## Calculation of VALUE 2 (I_NKP_max) 
I_Na_inf_calc = I_Na_inf(leakage_conductance, potential)
f_NaK_calc = f_NaK(params['resting_potential']*mvolt, params['V_T'])
Hill_Na = Hill(params['C0_Na_N'], params['zeta_Na'], 1.5)
Hill_K = Hill(params['C0_K_E'], params['zeta_K'], 1)

I_NKP_max = calculate_I_NKP_max(I_Na_inf_calc, f_NaK_calc, Hill_Na, Hill_K)

print(f'I_NKP_max: {I_NKP_max}')



#### CHECKS for error fixing

## CURRENT-CHECK_1 -> expected to be 0 as I_Na_inf = - 3 * I_NKP
CURRENT_CHECK_1 = I_Na_inf(leakage_conductance, potential) + 3 * I_NKP(I_NKP_max, f_NaK(potential, params['V_T']), params)
print(f'CURRENT CHECK 1: {CURRENT_CHECK_1}')

## CURRENT-CHECK_2 -> expected to be 0 as I_K_inf = 2 * I_NKP + I_KCC
CURRENT_CHECK_2 = I_K_inf(potential) - 2 * I_NKP(I_NKP_max, f_NaK(potential, params['V_T']), params) - I_KCC(params)
print(f'CURRENT CHECK 2: {CURRENT_CHECK_2}')

## CURRENT-CHECK_3 -> expected to be 0 as I_K_inf = 2 * I_NKP + I_KCC
CURRENT_CHECK_3 = I_Cl_inf(potential) + I_KCC(params)
print(f'CURRENT CHECK 3: {CURRENT_CHECK_3}')

## POTENIAL_CHECK -> expected to show that the calculated conductance matches the potential
POTENTIAL_CHECK = -I_KCC(params)/params['g_Cl_L'] + params['E_Cl']
print(f'POTENTIAL CHECK: {POTENTIAL_CHECK}')


## TOTAL CURRENT CHECK
## Check whether the total ionic current is 0 and EQUILIBRIUM is achieved
total_current = I_Na_inf(leakage_conductance, potential) + I_K_inf(potential) + I_Cl_inf(potential) + I_NKP(I_NKP_max, f_NaK(potential, params['V_T']), params)
print(f'TOTAL CURRENT: {total_current}')

print('NERNST Cl')
print(NernstPotential(params['C0_Cl_E'], params['C0_Cl_N'], -1, 37))
print('NERNST K')
print(NernstPotential(params['C0_K_E'], params['C0_K_N'], 1, 37))
print('Na_Nernst')
print(NernstPotential(params['C0_Na_E'], params['C0_Na_N'], 1, 37))

print(f'I_Na_inf {I_Na_inf_calc}')

I_K_CURRENT = I_K_inf(potential)
print(f'I_K: {I_K_CURRENT}')
print('n')
print(gating_variable_n(potential))
print(params['E_K'])

I_Cl_CURRENT = I_Cl_inf(potential)
print(f'I_Cl_CURRENT: {I_Cl_CURRENT}')

I_KCC_CURRENT = I_KCC(params)
print(f'I_KCC_CURRENT: {I_KCC_CURRENT}')

NKP_current = I_NKP(I_NKP_max, f_NaK(params['resting_potential'] * mvolt, params['V_T']), params)
print(f'NKP_CURRENT: {NKP_current}')


{'C0_Na_N': 10. * mmolar, 'C0_Na_E': 145. * mmolar, 'C0_K_N': 130. * mmolar, 'C0_K_E': 3. * mmolar, 'C0_Cl_N': 5. * mmolar, 'C0_Cl_E': 130. * mmolar, 'c_m': 0.01 * metre ** -4 * kilogram ** -1 * second ** 4 * amp ** 2, 'g_Na': 20400. * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_Na_L': nan * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_K_L': 0. * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_K': 6930. * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_Cl_L': 0.5 * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'g_KCC': 0.62544968 * metre ** -4 * kilogram ** -1 * second ** 3 * amp ** 2, 'v_thr': -50. * mvolt, 'resting_potential': -70, 'U_m': -38. * mvolt, 'U_h': -66. * mvolt, 'U_n': 18.7 * mvolt, 'W_m': 6. * mvolt, 'W_h': 6. * mvolt, 'W_n': 9.7 * mvolt, 'T_exp': 37, 'I_dc': 0. * metre ** -2 * amp, 'I_ramp': 0. * metre ** -2 * amp, 'I_max': 0.04 * metre ** -2 * amp, 'T_ramp': 100. * second, 'TimedArray': <brian2.input.timedarray.Tim